### Calculate Pgen for list of sequences. The finalised script is at the bottom of this page. 

In [ ]:
# First, install OLGA (if you haven’t already):
#pip install olga

from olga.load_model import read_model
from olga.generation import GenerationProbabilityVDJ
import numpy as np

# 1. Load a pre-trained TRB recombination model
#    You can download models from https://github.com/zsethna/OLGA/tree/master/data/models
model_dir = "/Users/ishaharris/Projects/TCR/TCR-Isha/tcr_env/lib/python3.11/site-packages/olga/default_models/human_T_beta"
model = read_model(model_dir)

# 2. Load data
highconf_dir  = '/Users/ishaharris/Projects/TCR/TCR-Isha/data/High_Confidence_CMV_TCR'
highconf_file = os.path.join(highconf_dir, 'WITHOUT_EBV_highconf.tsv')
highconf      = pd.read_csv(highconf_file, sep='\t')['amino_acid'].tolist()


# 3. Compute Pgen for each sequence
#    distribution.generate_Pgen_aa returns a float Pgen for a given aa sequence
pgen_list = []
for seq in highconf:
    p = distribution.generate_Pgen_aa(seq, model)
    pgen_list.append(p)

# 4. Report results
for seq, p in zip(highconf, pgen_list):
    print(f"Sequence: {seq:15s}  Pgen: {p:.3e}")

# 5. (Optional) Convert Pgens to log10 scale
log10_pgen = np.log10(pgen_list)
for seq, logp in zip(highconf, log10_pgen):
    print(f"Sequence: {seq:15s}  log10(Pgen): {logp:.2f}")


NameError: name 'read_model' is not defined

In [ ]:
## updated version below, 2024-06-26


# pip install olga pandas

import os
import pandas as pd
import numpy as np

from olga.load_model import GenomicDataVDJ, GenerativeModelVDJ
from olga.generation_probability import GenerationProbabilityVDJ

# --- Locate the packaged human TRB model directory in OLGA ---
try:
    from importlib.resources import files
    model_dir = str(files("olga") / "default_models" / "human_T_beta")
except Exception:
    import olga as _olga
    model_dir = os.path.join(os.path.dirname(_olga.__file__), "default_models", "human_T_beta")

# Helper: choose .csv if present, else .txt
def pick_anchor(model_dir: str, basename: str) -> str:
    cand_csv = os.path.join(model_dir, f"{basename}.csv")
    cand_txt = os.path.join(model_dir, f"{basename}.txt")
    if os.path.exists(cand_csv):
        return cand_csv
    if os.path.exists(cand_txt):
        return cand_txt
    raise FileNotFoundError(
        f"Neither {basename}.csv nor {basename}.txt found in {model_dir}. "
        f"Contents: {sorted(os.listdir(model_dir))}"
    )

params_file     = os.path.join(model_dir, "model_params.txt")
marginals_file  = os.path.join(model_dir, "model_marginals.txt")
V_anchor_file   = pick_anchor(model_dir, "V_gene_CDR3_anchors")
J_anchor_file   = pick_anchor(model_dir, "J_gene_CDR3_anchors")

# --- Load genomic data + model (IGoR-format assets) ---
genomic_data = GenomicDataVDJ()
genomic_data.load_igor_genomic_data(params_file, V_anchor_file, J_anchor_file)

generative_model = GenerativeModelVDJ()
generative_model.load_and_process_igor_model(marginals_file)

pgen_model = GenerationProbabilityVDJ(generative_model, genomic_data)

# Compute AA Pgen (unconstrained by V/J)
def pgen_aa(seq: str) -> float:
    try:
        return pgen_model.compute_aa_CDR3_pgen(seq)
    except TypeError:
        return pgen_model.compute_aa_CDR3_pgen(seq, None, None)

# --- Your original loading pattern ---
highconf_dir  = '/Users/ishaharris/Projects/TCR/TCR-Isha/data/High_Confidence_CMV_TCR'
highconf_file = os.path.join(highconf_dir, 'vdjdb.tsv')
highconf      = pd.read_csv(highconf_file, sep='\t')['CDR3'].astype(str).tolist()

# --- Compute Pgen for each sequence ---
pgen_list = []
for seq in highconf:
    p = pgen_aa(seq)
    pgen_list.append(p)

# --- Report results ---
for seq, p in zip(highconf, pgen_list):
    print(f"Sequence: {seq:15s}  Pgen: {p:.3e}")

# --- Optional: log10(Pgen), safe for zeros ---
log10_pgen = [(-np.inf if p <= 0 else np.log10(p)) for p in pgen_list]
for seq, logp in zip(highconf, log10_pgen):
    if np.isneginf(logp):
        print(f"Sequence: {seq:15s}  log10(Pgen): -inf (Pgen=0)")
    else:
        print(f"Sequence: {seq:15s}  log10(Pgen): {logp:.2f}")


KeyboardInterrupt: 

### This is the code that computes and saves the output - it is functional of 04/09/25.

In [14]:
import subprocess
import os
import pandas as pd
import numpy as np

# Load data
highconf_dir  = '/Users/ishaharris/Projects/TCR/TCR-Isha/data/crossreactors'
highconf_file = os.path.join(highconf_dir, 'vdjdb_random_total_burden.csv')
highconf      = pd.read_csv(highconf_file)['Sequence'].tolist()


def get_pgen(seq):
    cmd = ["olga-compute_pgen", "--humanTRB", seq]
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    if result.returncode != 0:
        print(f"Error running OLGA for {seq}: {result.stderr}")
        return None

    # Parse the line that contains the probability
    for line in result.stdout.splitlines():
        if "Pgen of the amino acid sequence" in line:
            try:
                pgen_str = line.split(":")[-1].strip()
                return float(pgen_str)
            except ValueError:
                print(f"Couldn't parse float from line: {line}")
                return None

    print(f"No Pgen found in output for {seq}")
    return None

# Compute and track progress
pgen_dict = {}  # or use a Series later
for i, seq in enumerate(highconf, 1):
    print(f"[{i}/{len(highconf)}] Computing Pgen for: {seq}")
    p = get_pgen(seq)
    if p is not None:
        pgen_dict[seq] = p
    else:
        print(f"Failed to compute Pgen for sequence: {seq}")


# Report results in input order
for seq in highconf:
    if seq in pgen_dict:
        p = pgen_dict[seq]
        print(f"Sequence: {seq:15s}  Pgen: {p:.3e}")
    else:
        print(f"Sequence: {seq:15s}  Pgen: FAILED")


# Save results
pgen_series = pd.Series(pgen_dict, name="Pgen")
pgen_series.index.name = "Sequence"


pgen_dir = '/Users/ishaharris/Projects/TCR/TCR-Isha/data/pgen'

output_path = os.path.join(pgen_dir, 'all_vatcr.tsv')
pgen_series.to_csv(output_path, sep='\t')

print(f"\nSaved Pgen values to: {output_path}")



[1/100] Computing Pgen for: CASSLGQGAYEQYF
[2/100] Computing Pgen for: CASSTGRYYEQYF
[3/100] Computing Pgen for: CASSYSQGAYEQYF
[4/100] Computing Pgen for: CASSYRANYEQYF
[5/100] Computing Pgen for: CASPGDLNTEAFF
[6/100] Computing Pgen for: CASSVAGSYNEQFF
[7/100] Computing Pgen for: CASSVYGGAEAFF
[8/100] Computing Pgen for: CASSPGLRPEAFF
[9/100] Computing Pgen for: CASSFGTVYNEQFF


KeyboardInterrupt: 

In [ ]:
# Report results in input order
for seq in highconf:
    if seq in pgen_dict:
        p = pgen_dict[seq]
        print(f"Sequence: {seq:15s}  Pgen: {p:.3e}")
    else:
        print(f"Sequence: {seq:15s}  Pgen: FAILED")


# Save results
pgen_series = pd.Series(pgen_dict, name="Pgen")
pgen_series.index.name = "Sequence"


pgen_dir = '/Users/ishaharris/Projects/TCR/TCR-Isha/data/pgen'

output_path = os.path.join(pgen_dir, 'donor5_IDH_pgen.tsv')
pgen_series.to_csv(output_path, sep='\t')

print(f"\nSaved Pgen values to: {output_path}")



Sequence: CASGPVDTDTQYF    Pgen: 2.003e-09
Sequence: CASSSLTYEQYF     Pgen: 4.256e-07
Sequence: CASSFGEKYGYTF    Pgen: 2.591e-09
Sequence: CSVAEMTGATEAFF   Pgen: 2.682e-12
Sequence: CASSLDGLAGGQGYNEQFF  Pgen: 1.025e-10
Sequence: CASSPGQRTDTQYF   Pgen: 8.180e-08
Sequence: CASSWGLAGGVRNEQFF  Pgen: 1.099e-10
Sequence: CASSSITGTSGHSTDTQYF  Pgen: 3.729e-12
Sequence: CASSYPRKDYGYTF   Pgen: 2.487e-09
Sequence: CASSSLPRRGQETQYF  Pgen: 4.481e-10
Sequence: CASSPRLAGEETQYF  Pgen: 4.957e-08
Sequence: CASIDRTNTGELFF   Pgen: 5.151e-09
Sequence: CASSWPFGGTNTGELFF  Pgen: 1.987e-11
Sequence: CASSEHGLAVDEQFF  Pgen: 6.779e-11
Sequence: CASSLGGKGRNTEAFF  Pgen: 3.507e-09
Sequence: CASSQDVQGAAEAFF  Pgen: 6.858e-10
Sequence: CASSMFGQGQPQHF   Pgen: 7.098e-10
Sequence: CASSVEAVYTEAFF   Pgen: 6.195e-10
Sequence: CASSLEQVMDEQYF   Pgen: 6.872e-11
Sequence: CASSQAWELRYEQYF  Pgen: 6.361e-11
Sequence: CASSGESGTSNYNEQFF  Pgen: 1.366e-11
Sequence: CASSSANYGYTF     Pgen: 3.977e-07
Sequence: CSVLAGLTYNEQFF   Pgen: 5.765